<a href="https://colab.research.google.com/github/momokaakiyama/cinema/blob/main/chapter5/colab/LDA_content.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/oreilly-japan/RecommenderSystems/blob/main/chapter5/colab/LDA_content.ipynb)

# Latent Dirichlet Allocation (LDA)(内容ベースフィルタリング)

In [36]:
# Colab用のnotebookです。このnotebook1枚でデータのダウンロードから、レコメンドまで完結するようになっています。（予測評価は含めていません。）
# MovieLensデータがまだダウンロードされてなければこのセルを実行して、ダウンロードしてください
# MovieLensデータの分析は、data_download.ipynbをご参照ください

#1 データのダウンロードと解凍
!wget -nc --no-check-certificate https://files.grouplens.org/datasets/movielens/ml-10m.zip -P ../data
!unzip -n ../data/ml-10m.zip -d ../data/

File ‘../data/ml-10m.zip’ already there; not retrieving.

Archive:  ../data/ml-10m.zip


In [37]:
#2 Movielensのデータの読み込み（データ量が多いため、読み込みに時間がかかる場合があります）
import pandas as pd

# movieIDとタイトル名のみ使用
m_cols = ['movie_id', 'title', 'genre']
movies = pd.read_csv('../data/ml-10M100K/movies.dat', names=m_cols, sep='::' , encoding='latin-1', engine='python')

# genreをlist形式で保持する
movies['genre'] = movies.genre.apply(lambda x:x.split('|'))


# ユーザが付与した映画のタグ情報の読み込み
t_cols = ['user_id', 'movie_id', 'tag', 'timestamp']
user_tagged_movies = pd.read_csv('../data/ml-10M100K/tags.dat', names=t_cols, sep='::', engine='python')

# tagを小文字にする
user_tagged_movies['tag'] = user_tagged_movies['tag'].str.lower()


# tagを映画ごとにlist形式で保持する
movie_tags = user_tagged_movies.groupby('movie_id').agg({'tag':list})

# タグ情報を結合する
movies = movies.merge(movie_tags, on='movie_id', how='left')

# 評価値データの読み込み
r_cols = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('../data/ml-10M100K/ratings.dat', names=r_cols, sep='::', engine='python')


# データ量が多いため、ユーザー数を1000に絞って、試していく
valid_user_ids = sorted(ratings.user_id.unique())[:1000]
ratings = ratings[ratings["user_id"].isin(valid_user_ids)]


# 映画のデータと評価のデータを結合する
movielens = ratings.merge(movies, on='movie_id')

print(f'unique_users={len(movielens.user_id.unique())}, unique_movies={len(movielens.movie_id.unique())}')

# 学習用とテスト用にデータを分割する
# 各ユーザの直近の５件の映画を評価用に使い、それ以外を学習用とする
# まずは、それぞれのユーザが評価した映画の順序を計算する
# 直近付与した映画から順番を付与していく(1始まり)

movielens['timestamp_rank'] = movielens.groupby(
    'user_id')['timestamp'].rank(ascending=False, method='first')
movielens_train = movielens[movielens['timestamp_rank'] > 5]
movielens_test = movielens[movielens['timestamp_rank']<= 5]

unique_users=1000, unique_movies=6736


In [38]:
#3 LDAの学習に必要なパラメタを定義
# 因子数(トピック数)
factors = 50
# エポック数
n_epochs = 30

In [39]:
movie_content = movies.copy()
# tagが付与されていない映画もあるが、genreはすべての映画に付与されている
# tagとgenreを結合したものを映画のコンテンツ情報として似ている映画を探して推薦していく
# tagがない映画に関しては、NaNになっているので、空のリストに変換してから処理をする
movie_content["tag_genre"] = movie_content["tag"].fillna("").apply(list) + movie_content["genre"].apply(list)
movie_content["tag_genre"] = movie_content["tag_genre"].apply(lambda x: list(map(str, x)))

#4 タグとジャンルデータを使って、ldaを学習する
tag_genre_data = movie_content.tag_genre.tolist()

In [40]:
#5 ColabのPythonにgensimをインストールする
import sys
!{sys.executable} -m pip install gensim

In [41]:
from gensim.corpora.dictionary import Dictionary

#6 LDAのインプットに使うデータの作成
common_dictionary = Dictionary(tag_genre_data)
common_corpus = [common_dictionary.doc2bow(text) for text in tag_genre_data]

In [42]:
import gensim

#7 LDAの学習（潜在トピックを洗い出す）
lda_model = gensim.models.LdaModel(
    common_corpus, id2word=common_dictionary, num_topics=factors, passes=n_epochs
)

In [43]:
# (参考)topic0の単語一覧（LDAが見つけた潜在トピックを構成する単語を確認）
for token_id, score in lda_model.get_topic_terms(0, topn=10):
    word = common_dictionary.id2token[token_id]
    print(f'word={word}, score={score}')

word=bibliothek, score=0.10934405028820038
word=Drama, score=0.07876186072826385
word=sports, score=0.049390438944101334
word=library vhs, score=0.03238710016012192
word=on dvr, score=0.03159065544605255
word=keira knightley, score=0.030789537355303764
word=eric's dvds, score=0.027269233018159866
word=hw drama, score=0.026678813621401787
word=jude law, score=0.026579832658171654
word=seen, score=0.02614528499543667


In [44]:
# (参考)サムライという単語のトピック(各トピックの所属確率)
lda_model[common_dictionary.doc2bow(['samurai'])]

[(0, np.float32(0.010000044)),
 (1, np.float32(0.010000044)),
 (2, np.float32(0.010000044)),
 (3, np.float32(0.010000044)),
 (4, np.float32(0.010000044)),
 (5, np.float32(0.010000044)),
 (6, np.float32(0.010000044)),
 (7, np.float32(0.010000044)),
 (8, np.float32(0.010000044)),
 (9, np.float32(0.010000044)),
 (10, np.float32(0.010000044)),
 (11, np.float32(0.010000044)),
 (12, np.float32(0.010000044)),
 (13, np.float32(0.010000044)),
 (14, np.float32(0.010000044)),
 (15, np.float32(0.010000044)),
 (16, np.float32(0.010000044)),
 (17, np.float32(0.010000044)),
 (18, np.float32(0.010000044)),
 (19, np.float32(0.010000044)),
 (20, np.float32(0.010000044)),
 (21, np.float32(0.010000044)),
 (22, np.float32(0.010000044)),
 (23, np.float32(0.010000044)),
 (24, np.float32(0.010000044)),
 (25, np.float32(0.010000044)),
 (26, np.float32(0.010000044)),
 (27, np.float32(0.010000044)),
 (28, np.float32(0.010000044)),
 (29, np.float32(0.010000044)),
 (30, np.float32(0.010000044)),
 (31, np.float32(0

In [45]:
#8 各映画のトピックを格納
lda_topics = lda_model[common_corpus]

#9 各映画に最も確率の高いトピックを１つ取り出し格納していく
movie_topics = []
movie_topic_scores = []
for movie_index, lda_topic in enumerate(lda_topics):
    sorted_topic = sorted(lda_topics[movie_index], key=lambda x: -x[1])
    # 最も確率の高いトピック
    movie_topic, topic_score = sorted_topic[0]
    movie_topics.append(movie_topic)
    movie_topic_scores.append(topic_score)
movie_content["topic"] = movie_topics
movie_content["topic_score"] = movie_topic_scores
movie_content

,movie_id,title,genre,tag,tag_genre,topic,topic_score
0,1,Toy Story (1995),"[Adventure, Animation, Children, Comedy, Fantasy]","[pixar, pixar, pixar, animation, pixar, animat...","[pixar, pixar, pixar, animation, pixar, animat...",11,0.528782
1,2,Jumanji (1995),"[Adventure, Children, Fantasy]","[for children, game, animals, joe johnston, ro...","[for children, game, animals, joe johnston, ro...",4,0.269334
2,3,Grumpier Old Men (1995),"[Comedy, Romance]","[funniest movies, comedinha de velhinhos engra...","[funniest movies, comedinha de velhinhos engra...",25,0.418297
3,4,Waiting to Exhale (1995),"[Comedy, Drama, Romance]",[girl movie],"[girl movie, Comedy, Drama, Romance]",22,0.755000
4,5,Father of the Bride Part II (1995),[Comedy],"[steve martin, pregnancy, remake, steve martin...","[steve martin, pregnancy, remake, steve martin...",11,0.457888
...,...,...,...,...,...,...,...
10676,65088,Bedtime Stories (2008),"[Adventure, Children, Comedy]",NaN,"[Adventure, Children, Comedy]",4,0.755000
10677,65091,Manhattan Melodrama (1934),"[Crime, Drama, Romance]",NaN,"[Crime, Drama, Romance]",22,0.450531
10678,65126,Choke (2008),"[Comedy, Drama]","[chuck palahniuk, based on book]","[chuck palahniuk, based on book, Comedy, Drama]",22,0.505006
10679,65130,Revolutionary Road (2008),"[Drama, Romance]",[toplist08],"[toplist08, Drama, Romance]",22,0.673333


In [46]:
from collections import defaultdict
from collections import Counter

#10 各ユーザーのレコメンドリストを作成していく
# ユーザーが高く評価した映画が、どのトピックに所属していることが多いかをカウントする
# 一番多いトピックをユーザーの好きなトピックとみなして、そのトピックの映画をおすすめする

movielens_train_high_rating = movielens_train[movielens_train.rating >= 4]
user_evaluated_movies = movielens_train.groupby("user_id").agg({"movie_id": list})["movie_id"].to_dict()

movie_id2index = dict(zip(movie_content.movie_id.tolist(), range(len(movie_content))))
pred_user2items = defaultdict(list)
for user_id, data in movielens_train_high_rating.groupby("user_id"):
    # ユーザーが高く評価した映画
    evaluated_movie_ids = user_evaluated_movies[user_id]
    # 直近閲覧した映画を取得
    movie_ids = data.sort_values("timestamp")["movie_id"].tolist()[-10:]

    movie_indexes = [movie_id2index[id] for id in movie_ids]

    # 直近閲覧した映画のトピックを取得して、出現回数をカウントする
    topic_counter = Counter([movie_topics[i] for i in movie_indexes])
    # 一番出現回数が多かったトピックを取得
    frequent_topic = topic_counter.most_common(1)[0][0]
    # そのトピックの映画の中でもスコアが高いものをおすすめする
    topic_movies = (
        movie_content[movie_content.topic == frequent_topic]
        .sort_values("topic_score", ascending=False)
        .movie_id.tolist()
    )

    for movie_id in topic_movies:
        if movie_id not in evaluated_movie_ids:
            pred_user2items[user_id].append(movie_id)
        if len(pred_user2items[user_id]) == 10:
            break

In [47]:
#11 User 2 が高く評価した映画と、それらに割り当てられたトピック
user2_high_rated_movies = movielens_train_high_rating[movielens_train_high_rating.user_id==2]
display(user2_high_rated_movies.merge(movie_content[['movie_id', 'topic', 'topic_score']], on='movie_id', how='left'))

,user_id,movie_id,rating,timestamp,title,genre,tag,timestamp_rank,topic,topic_score
0,2,110,5.0,868245777,Braveheart (1995),"[Action, Drama, War]","[bullshit history, medieval, bloodshed, hero, ...",8.0,26,0.518436
1,2,260,5.0,868244562,Star Wars: Episode IV - A New Hope (a.k.a. Sta...,"[Action, Adventure, Sci-Fi]","[desert, quotable, lucas, gfei own it, seen mo...",17.0,47,0.813741
2,2,590,5.0,868245608,Dances with Wolves (1990),"[Adventure, Drama, Western]","[afi 100, lame, native, biopic, american india...",11.0,35,0.330788
3,2,1210,4.0,868245644,Star Wars: Episode VI - Return of the Jedi (1983),"[Action, Adventure, Sci-Fi]","[desert, fantasy, sci-fi, space, lucas, gfei o...",10.0,47,0.857472


In [48]:
#12 上記の結果から、user_id=2はtopic47の映画が一番多く評価しているため、システムはtopic47に属する未評価の映画を推薦する。
#User2に推薦された映画と、それらに割り当てられたトピック
user2_recommended_movie_ids = pred_user2items[2]
user2_recommended_movies = movies[movies.movie_id.isin(user2_recommended_movie_ids)]
display(user2_recommended_movies.merge(movie_content[['movie_id', 'topic', 'topic_score']], on='movie_id', how='left').sort_values('topic_score', ascending=False))

,movie_id,title,genre,tag,topic,topic_score
2,1196,Star Wars: Episode V - The Empire Strikes Back...,"[Action, Adventure, Sci-Fi]","[lucas, george lucas, george lucas, gfei own i...",47,0.859588
7,5378,Star Wars: Episode II - Attack of the Clones (...,"[Action, Adventure, Sci-Fi]","[lucas, dvd, seen at the cinema, nerdy, sci-fi...",47,0.807566
9,33493,Star Wars: Episode III - Revenge of the Sith (...,"[Action, Adventure, Fantasy, Sci-Fi]","[lucas, divx, space, space opera, space, on co...",47,0.794908
4,2628,Star Wars: Episode I - The Phantom Menace (1999),"[Action, Adventure, Sci-Fi]","[far future, lucas, fantasy, george lucas, luc...",47,0.775019
3,2528,Logan's Run (1976),"[Action, Adventure, Sci-Fi]","[utopia, futuristmovies.com, nudity (topless),...",47,0.753446
1,1129,Escape from New York (1981),"[Action, Adventure, Sci-Fi, Thriller]","[chaos, future, prison, futuristmovies.com, bi...",47,0.751692
5,2808,Universal Soldier (1992),"[Action, Sci-Fi]","[futuristmovies.com, cyborgs, cyborgs, seen at...",47,0.723481
6,2986,RoboCop 2 (1990),"[Action, Crime, Sci-Fi]","[robots, first robots, android(s)/cyborg(s), c...",47,0.666744
0,176,Living in Oblivion (1995),[Comedy],"[behind-the-scenes, film-making, meta-indie, b...",47,0.637938
8,8644,"I, Robot (2004)","[Action, Adventure, Sci-Fi, Thriller]","[based on a book, isaac asimov, action, fight,...",47,0.609912


このように、推薦された映画の多くも`topic`25に属していることが確認できる。これが、コンテンツベースレコメンデーションの動作原理である。ユーザーが高く評価したコンテンツの主題（トピック）を特定し、その主題に合致する新しいコンテンツを推薦している。

In [49]:
#13 今回作成したuser_id=-1のデータにtopicとtopic_scoreをマージした
import pandas as pd

user_ratings_df = pd.read_csv('/content/myselfdata.csv')
display(user_ratings_df.merge(movie_content[['movie_id', 'topic', 'topic_score']], on='movie_id', how='left'))

,user_id,movie_id,rating,timestamp,title,genre,tag,timestamp_rank,topic,topic_score
0,-1,4896,5.0,1.770030e+09,Harry Potter and the Sorcerer's Stone (a.k.a. ...,"['Adventure', 'Children', 'Fantasy']","['based on a book', 'franchise', 'adventure', ...",1,37,0.326538
1,-1,5816,5.0,1.770030e+09,Harry Potter and the Chamber of Secrets (2002),"['Adventure', 'Children', 'Fantasy']","['based on a book', 'franchise', 'harry potter...",2,37,0.436823
2,-1,8368,4.0,1.770030e+09,Harry Potter and the Prisoner of Azkaban (2004),"['Adventure', 'Children', 'Fantasy']","['based on a book', 'franchise', 'magic', 'har...",3,37,0.488076
3,-1,40815,4.5,1.770030e+09,Harry Potter and the Goblet of Fire (2005),"['Adventure', 'Fantasy', 'Thriller']","['based on a book', 'big budget', 'franchise',...",4,37,0.429550
4,-1,54001,0.5,1.770030e+09,Harry Potter and the Order of the Phoenix (2007),"['Adventure', 'Drama', 'Fantasy', 'IMAX']","['franchise', 'too short', 'alan rickman', 'br...",5,37,0.552768
5,-1,5971,3.0,1.770030e+09,My Neighbor Totoro (Tonari no Totoro) (1988),"['Animation', 'Children', 'Fantasy']","['anime', 'miyazaki', 'anime', 'hayao miyazaki...",6,18,0.409133
6,-1,5618,5.0,1.770030e+09,Spirited Away (Sen to Chihiro no kamikakushi) ...,"['Adventure', 'Animation', 'Children', 'Fantasy']","['anime', ""holy christ it's fantastic in every...",7,18,0.329655
7,-1,1022,4.5,1.770030e+09,Cinderella (1950),"['Animation', 'Children', 'Fantasy', 'Musical'...","['disney', 'disney', 'library vhs', 'fairy tal...",8,4,0.820111
8,-1,2125,3.0,1.770030e+09,Ever After: A Cinderella Story (1998),"['Comedy', 'Drama', 'Fantasy', 'Romance']","['definitely for girls', 'lovely', 'xvgb', 'fu...",9,6,0.237314
9,-1,63239,5.0,1.770030e+09,Cinderella (1997),"['Children', 'Fantasy', 'Musical', 'Romance']",NaN,10,4,0.604000


In [50]:
#14 user=-1のレコメンドリストを作成していく
# ユーザーが高く評価した映画が、どのトピックに所属していることが多いかをカウントする
# 一番多いトピックをユーザーの好きなトピックとみなして、そのトピックの映画をおすすめする

user_ratings_df_high_rated = user_ratings_df[user_ratings_df['rating'] >= 4.0]

movie_id2index = dict(zip(movie_content.movie_id.tolist(), range(len(movie_content))))
movie_indexes = [movie_id2index[id] for id in user_ratings_df_high_rated['movie_id']]
# 直近閲覧した映画のトピックを取得して、出現回数をカウントする
topic_counter = Counter([movie_topics[i] for i in movie_indexes])
# 一番出現回数が多かったトピックを取得
frequent_topic = topic_counter.most_common(1)[0][0]
# そのトピックの映画の中でもスコアが高いものをおすすめする
topic_movies = (
    movie_content[movie_content.topic == frequent_topic]
    .sort_values("topic_score", ascending=False)
    .movie_id.tolist()
)

pred_user2items = defaultdict(list)
for movie_id in topic_movies:
    if movie_id not in evaluated_movie_ids:
        pred_user2items[-1].append(movie_id)

In [51]:
# 15　推薦された映画の詳細データを抽出
#user_id=-1はtopic37の映画を一番多く評価しているため、システムはtopic37に属する未評価の映画を推薦する。
recommended_movies = movie_content[
    movie_content['movie_id'].isin(pred_user2items[-1])
]

display(recommended_movies[['movie_id', 'title', 'genre', 'topic', 'topic_score']])

,movie_id,title,genre,topic,topic_score
16,17,Sense and Sensibility (1995),"[Comedy, Drama, Romance]",37,0.633623
27,28,Persuasion (1995),"[Drama, Romance]",37,0.721389
38,39,Clueless (1995),"[Comedy, Romance]",37,0.265355
80,81,Things to Do in Denver When You're Dead (1995),"[Crime, Drama, Romance]",37,0.199989
143,145,Bad Boys (1995),[Action],37,0.357122
...,...,...,...,...,...
10347,60421,"Stone Angel, The (2007)",[Drama],37,0.604009
10468,62049,1984 (1956),"[Drama, Sci-Fi]",37,0.352141
10470,62063,Dead Man's Letters (Pisma myortvogo cheloveka)...,"[Drama, Film-Noir]",37,0.673333
10542,63121,"Tale of Two Cities, A (1958)","[Drama, Romance]",37,0.370439


In [52]:
#(参考) 上記user_id=-1の学習データで、4以上の評価を付けた映画一覧
display(user_ratings_df[user_ratings_df['rating'] >= 4.0])

,user_id,movie_id,rating,timestamp,title,genre,tag,timestamp_rank
0,-1,4896,5.0,1.770030e+09,Harry Potter and the Sorcerer's Stone (a.k.a. ...,"['Adventure', 'Children', 'Fantasy']","['based on a book', 'franchise', 'adventure', ...",1
1,-1,5816,5.0,1.770030e+09,Harry Potter and the Chamber of Secrets (2002),"['Adventure', 'Children', 'Fantasy']","['based on a book', 'franchise', 'harry potter...",2
2,-1,8368,4.0,1.770030e+09,Harry Potter and the Prisoner of Azkaban (2004),"['Adventure', 'Children', 'Fantasy']","['based on a book', 'franchise', 'magic', 'har...",3
3,-1,40815,4.5,1.770030e+09,Harry Potter and the Goblet of Fire (2005),"['Adventure', 'Fantasy', 'Thriller']","['based on a book', 'big budget', 'franchise',...",4
6,-1,5618,5.0,1.770030e+09,Spirited Away (Sen to Chihiro no kamikakushi) ...,"['Adventure', 'Animation', 'Children', 'Fantasy']","['anime', ""holy christ it's fantastic in every...",7
7,-1,1022,4.5,1.770030e+09,Cinderella (1950),"['Animation', 'Children', 'Fantasy', 'Musical'...","['disney', 'disney', 'library vhs', 'fairy tal...",8
9,-1,63239,5.0,1.770030e+09,Cinderella (1997),"['Children', 'Fantasy', 'Musical', 'Romance']",NaN,10
10,-1,594,4.0,1.770030e+09,Snow White and the Seven Dwarfs (1937),"['Animation', 'Children', 'Drama', 'Fantasy', ...","['disney', 'classic', 'disney', 'national film...",11
14,-1,595,4.0,1.770030e+09,Beauty and the Beast (1991),"['Animation', 'Children', 'Fantasy', 'Musical'...","['fairy tale', 'disney', 'disney', 'disney', '...",15
18,-1,916,5.0,1.770030e+09,Roman Holiday (1953),"['Comedy', 'Romance']","[""good in it's day"", 'classic', 'audrey hepbur...",19
